In [2]:
import pandas as pd
import re
import requests
import os

from dotenv import load_dotenv

#.env 파일 로드
load_dotenv()
kakao_api_key = os.getenv("kakao_api_key")


## API 테스트

In [175]:
# 주택명 확인

df = pd.read_csv('병합_청약매물_목록_정보.csv', encoding='cp949')
df = df.drop_duplicates(subset="공고번호", keep='first').reset_index(drop=True)
df.head()

/var/folders/pl/c9208lms3f937xrsflvwrm640000gn/T/ipykernel_82380/2490788004.py:3: DtypeWarning: Columns (36,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('병합_청약매물_목록_정보.csv', encoding='cp949')


,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,...,모집공고홈페이지주소,주택형,공급세대수,순위,거주지역,접수건수,경쟁률,최저당첨가점,최고당첨가점,평균당첨가점
0,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,043.7207J,5,1순위,해당지역,0,(△5),NaN,NaN,NaN
1,2025000032,2025000032,성남판교 산운마을9단지 대방노블랜드 아파트 우선 분양전환 후 잔여세대(15세대),1,APT,3,국민,0,분양주택,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.9300,4,1순위,해당지역,57,57.00,NaN,NaN,NaN
2,2025000020,2025000020,e편한세상 제물포역 파크메종(조합원 취소분),1,APT,1,민영,0,분양주택,400,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.9393A,5,1순위,해당지역,13,2.60,34,49,41.5
3,2025000014,2025000014,평택청북 사랑으로 부영 2차 아파트 우선분양 전환 후 잔여세대,1,APT,1,민영,0,분양주택,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.9605,39,1순위,해당지역,9,(△3),0,0,0
4,2025000013,2025000013,고양창릉지구 S-6블록 공공분양주택(본청약),1,APT,3,국민,0,분양주택,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.0000O,17,1순위,해당지역,0,NaN,NaN,NaN,NaN


In [176]:
# 주택명 정제

import pandas as pd
import re

def preprocess_apartname(df):

    # 특정 키워드 리스트 (정확한 일치 단어 제거)
    custom_stopwords = {}

    # 특정 단어를 포함하는 단어 제거 (예: '지구'가 포함된 단어 제거)
    remove_if_contains = ['우선' , '분양' ,'후', '잔여세대', '모집공고', '공급', '세대', '-', '전환', '블록', '공공']

    # ✅ 괄호와 괄호 안의 텍스트 모두 제거하는 함수
    def remove_parentheses(text):
        """괄호와 괄호 안의 텍스트 모두 제거"""
        return re.sub(r'\(.*?\)', '', text).strip()

    # ✅ 필터링 조건 함수
    def filter_conditions(word):
        """단어가 제거 대상인지 확인"""
        # if re.search(r'[A-Za-z]', word):  # 영어 포함 단어 제거
        #     return False
        if word in custom_stopwords:  # 특정 키워드 제거
            return False
        if any(sub in word for sub in remove_if_contains):  # 특정 단어 포함 단어 제거
            return False
        return True  # 모든 조건을 통과하면 유지

    # ✅ 정제 함수
    def clean_name(address):
        # 괄호와 괄호 안의 텍스트 제거
        address = remove_parentheses(address)

        words = address.split()  # 띄어쓰기 기준으로 분리
        cleaned_words = [word for word in words if filter_conditions(word)]  # 필터링 적용
        cleaned_address = ' '.join(cleaned_words)  # 정제된 단어들 다시 합치기

        # 쉼표가 있으면 첫 번째 단어만 반환
        if ',' in cleaned_address:
            return cleaned_address.split(',')[0]
        return cleaned_address  # 쉼표가 없으면 그대로 반환

    # ✅ 주소 정제 적용
    df['정제된주택명'] = df['공급지역명'] + ' ' + df['주택명'].apply(clean_name)

    return df

In [172]:
df = preprocess_apartname(df)
df['정제된주택명'].tolist()

['경기 부천 JY 포에시아',
 '경기 성남판교 산운마을9단지 대방노블랜드 아파트',
 '인천 e편한세상 제물포역 파크메종',
 '경기 평택청북 사랑으로 부영 2차 아파트',
 '경기 고양창릉지구',
 '경기 고양창릉지구',
 '서울 래미안 원페를라',
 '경기 부천 중앙하이츠 심곡 102동',
 '경기 양주 용암 영무 예다음 더퍼스트',
 '인천 인천강화서희스타힐스 1단지',
 '인천 래미안 센트리폴',
 '인천 래미안 센트리폴',
 '경기 의왕 월암지구 대방 디에트르 레이크파크',
 '서울 더샵 퍼스트월드',
 '경기 의정부역 다채로움',
 '경기 덕소역 라온프라이빗 리버포레',
 '경기 평택 고덕국제신도시 미래도 파밀리에',
 '경기 브레인시티 푸르지오',
 '경기 곤지암역 센트럴 아이파크',
 '경기 평택 브레인시티 수자인',
 '서울 창경궁 롯데캐슬 시그니처',
 '인천 e편한세상 동인천 베이프런트',
 '서울 아크로 리츠카운티',
 '서울 힐스테이트 등촌역',
 '서울 한신더휴 하이엔에듀포레',
 '인천 인하대역 푸르지오 에듀포레',
 '경기 화성 비봉지구 금성백조 예미지2차',
 '경기 더 플래티넘 스카이헤론',
 '서울 서울원 아이파크',
 '서울 e편한세상 당산 리버파크',
 '경기 평촌자이 퍼스니티',
 '경기 군포대야미 대방 디에트르 시그니처',
 '경기 아크로 베스티뉴',
 '경기 e편한세상 일산 메이포레 3단지',
 '경기 문산역3차 동문 디 이스트 센트럴',
 '경기 e편한세상 일산 메이포레 1단지',
 '경기 평택 브레인시티 한신더휴',
 '인천 검단신도시 푸르지오 더 파크',
 '경기 곤지암역 제일풍경채',
 '경기 이천 중리지구 신안인스빌 퍼스티지',
 '경기 광명 유승한내들 라포레',
 '경기 덕계역 한신더휴 포레스트',
 '서울 VIORR',
 '경기 힐스테이트 평택역센트럴시티',
 '경기 의왕고천지구 대방 디에트르 센트럴',
 '경기 양주역 푸르지오 센터파크',
 '경기 오산세교 힐데스하임',
 '경기 오산세교 우미 린 센

In [173]:

def add_address_by_apartname(apartname):
  kakao_api_url = "https://dapi.kakao.com/v2/local/search/keyword.json"
  headers = {"Authorization": f"KakaoAK {kakao_api_key}"}

  response = requests.get(kakao_api_url, headers=headers, params={"query": apartname})
  data = response.json()['documents']
  has_data = len(data) != 0
  
  if has_data:
    address = data[0]['address_name']
    return address
  
  return ''

add_address_by_apartname('경기도 의왕시 월암동')

'경기 의왕시 월암동 543-3'

In [174]:
df['공급위치 by 주택명'] = df['정제된주택명'].apply(add_address_by_apartname)

KeyboardInterrupt: 

In [87]:
df.to_csv('공급위치 by 주택명.csv', index=False, encoding='cp949')

In [177]:
df_1 = pd.read_csv('공급위치 by 주택명.csv', encoding='cp949')
df_1[['공고번호', '주택명', '공급위치', '공급위치 by 주택명']]

,공고번호,주택명,공급위치,공급위치 by 주택명
0,2025000043,부천 JY 포에시아,경기도 부천시 원미구 신흥로 20,경기 부천시 원미구 심곡동 465-11
1,2025000032,성남판교 산운마을9단지 대방노블랜드 아파트 우선 분양전환 후 잔여세대(15세대),"경기도 성남시 분당구 산운로 126 (운중동, 산운마을)",NaN
2,2025000020,e편한세상 제물포역 파크메종(조합원 취소분),인천광역시 미추홀구 숭의동 18번지 일대,인천 미추홀구 숭의동 18-1
3,2025000014,평택청북 사랑으로 부영 2차 아파트 우선분양 전환 후 잔여세대,경기도 평택시 청북읍 청북남로 277 (부영사랑으로),경기 평택시 청북읍 옥길리 1104
4,2025000013,고양창릉지구 S-6블록 공공분양주택(본청약),"경기도 고양시 덕양구 원흥동, 동산동, 용두동 외 6개동 일원",경기 고양시 덕양구 도내동 548
...,...,...,...,...
848,2020000303,수원호매실14단지 B-3BL,"경기도 수원시 권선구 호매실로218번길 22 (호매실동, 호매실마을 14단지 벨섬시티)",경기 수원시 권선구 호매실동 1380
849,2020000302,수원호매실15단지 B-7BL,"경기도 수원시 권선구 호매실로165번길 70 (호매실동, 수원호매실휴먼시아15단지아파트)",경기 수원시 권선구 호매실동 1388
850,2020000301,수원호매실5단지 B-6BL,"경기도 수원시 권선구 금곡로 46 (금곡동, 수원호매실휴먼시아5단지아파트)",경기 수원시 권선구 금곡동 1081
851,2020000304,수원호매실13단지 A-4BL,"경기도 수원시 권선구 호매실로218번길 111 (호매실동, 호매실마을13단지)",경기 수원시 권선구 호매실동 1371


In [178]:
df['공급위치 by 주택명'] = df_1['공급위치 by 주택명']
df

,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,...,주택형,공급세대수,순위,거주지역,접수건수,경쟁률,최저당첨가점,최고당첨가점,평균당첨가점,공급위치 by 주택명
0,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,043.7207J,5,1순위,해당지역,0,(△5),NaN,NaN,NaN,경기 부천시 원미구 심곡동 465-11
1,2025000032,2025000032,성남판교 산운마을9단지 대방노블랜드 아파트 우선 분양전환 후 잔여세대(15세대),1,APT,3,국민,0,분양주택,410,...,059.9300,4,1순위,해당지역,57,57.00,NaN,NaN,NaN,NaN
2,2025000020,2025000020,e편한세상 제물포역 파크메종(조합원 취소분),1,APT,1,민영,0,분양주택,400,...,059.9393A,5,1순위,해당지역,13,2.60,34,49,41.5,인천 미추홀구 숭의동 18-1
3,2025000014,2025000014,평택청북 사랑으로 부영 2차 아파트 우선분양 전환 후 잔여세대,1,APT,1,민영,0,분양주택,410,...,059.9605,39,1순위,해당지역,9,(△3),0,0,0,경기 평택시 청북읍 옥길리 1104
4,2025000013,2025000013,고양창릉지구 S-6블록 공공분양주택(본청약),1,APT,3,국민,0,분양주택,410,...,059.0000O,17,1순위,해당지역,0,NaN,NaN,NaN,NaN,경기 고양시 덕양구 도내동 548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,2020000303,2020000303,수원호매실14단지 B-3BL,1,APT,3,국민,1,분양전환 가능임대,410,...,075.8400A,20.0,1순위,해당지역,0,NaN,NaN,NaN,NaN,경기 수원시 권선구 호매실동 1380
849,2020000302,2020000302,수원호매실15단지 B-7BL,1,APT,3,국민,1,분양전환 가능임대,410,...,074.9300G,50.0,1순위,해당지역,0,NaN,NaN,NaN,NaN,경기 수원시 권선구 호매실동 1388
850,2020000301,2020000301,수원호매실5단지 B-6BL,1,APT,3,국민,1,분양전환 가능임대,410,...,074.9300,10.0,1순위,해당지역,0,NaN,NaN,NaN,NaN,경기 수원시 권선구 금곡동 1081
851,2020000304,2020000304,수원호매실13단지 A-4BL,1,APT,3,국민,1,분양전환 가능임대,410,...,059.8100B,10.0,1순위,해당지역,0,NaN,NaN,NaN,NaN,경기 수원시 권선구 호매실동 1371


In [179]:
import pandas as pd
import re

def preprocess_address(df):
    # 특정 키워드 리스트 (정확한 일치 단어 제거)
    custom_stopwords = {'내', '외'}

    # 특정 단어를 포함하는 단어 제거 (예: '지구'가 포함된 단어 제거)
    remove_if_contains = ['지구', '사업', '일원', '일대', '블록', '공동', '일반', '개발', '필지', '구역', '역세권', '블럭', '주상복합', '시티', '신도시', '종전', '부동산']

    # ✅ 괄호와 괄호 안의 텍스트 모두 제거하는 함수
    def remove_parentheses(text):
        """괄호와 괄호 안의 텍스트 모두 제거"""
        return re.sub(r'\(.*?\)', '', text).strip()

    # ✅ 필터링 조건 함수
    def filter_conditions(word):
        """단어가 제거 대상인지 확인"""
        if re.search(r'[A-Za-z]', word):  # 영어 포함 단어 제거
            return False
        if word in custom_stopwords:  # 특정 키워드 제거
            return False
        if any(sub in word for sub in remove_if_contains):  # 특정 단어 포함 단어 제거
            return False
        return True  # 모든 조건을 통과하면 유지

    # ✅ 정제 함수
    def clean_address(address):
        # 괄호와 괄호 안의 텍스트 제거
        address = remove_parentheses(address)

        words = address.split()  # 띄어쓰기 기준으로 분리
        cleaned_words = [word for word in words if filter_conditions(word)]  # 필터링 적용
        cleaned_address = ' '.join(cleaned_words)  # 정제된 단어들 다시 합치기

        # 쉼표가 있으면 첫 번째 단어만 반환
        if ',' in cleaned_address:
            return cleaned_address.split(',')[0]
        return cleaned_address  # 쉼표가 없으면 그대로 반환

    # ✅ 주소 정제 적용
    df['정제된주소'] = df['공급위치'].apply(clean_address)

    return df

In [180]:
df = preprocess_address(df)
df['정제된주소'].tolist()

['경기도 부천시 원미구 신흥로 20',
 '경기도 성남시 분당구 산운로 126',
 '인천광역시 미추홀구 숭의동 18번지',
 '경기도 평택시 청북읍 청북남로 277',
 '경기도 고양시 덕양구 원흥동',
 '경기도 고양시 덕양구 원흥동',
 '서울특별시 서초구 방배동 818-14번지',
 '경기도 부천시 심곡동 803번지',
 '경기도 양주시 은현면 용암리 784-8번지',
 '인천광역시 강화군 선원면 창리456번지',
 '인천광역시 연수구 옥련동',
 '인천광역시 연수구 옥련동',
 '경기도 의왕시 월암동',
 '서울특별시 중랑구 상봉동 83-1번지',
 '경기도 의정부시 의정부동 94-8',
 '경기도 남양주시 와부읍 덕소리 453-15',
 '경기도 평택시 고덕동',
 '경기도 평택시 장안동',
 '경기도 광주시 곤지암읍 곤지암리 637번지',
 '경기도 평택시',
 '서울특별시 성북구 삼선동2가 296번지',
 '인천광역시 중구 사동 23-4번지',
 '서울특별시 서초구 방배동 1018-1번지',
 '서울특별시 강서구 등촌동 366-24번지',
 '서울특별시 금천구 시흥동 220-2외',
 '인천광역시 미추홀구 학익동 321번지',
 '경기도 화성시 화성비봉',
 '경기도 평택시 통복동 108-3번지',
 '서울특별시 노원구 월계동 85-7번지',
 '서울특별시 영등포구 당산동4가 91번지',
 '경기도 안양시 동안구 비산3동 354-10번지',
 '경기도 군포시 군포대야미',
 '경기도 안양시 동안구 호계동 915번지',
 '경기도 고양시 일산동구 풍동 852번지',
 '경기도 파주시 문산읍 문산리 81-61번지',
 '경기도 고양시 일산동구 풍동 822번지',
 '경기도 평택시 장안동',
 '인천광역시 서구',
 '경기도 광주시 곤지암읍 곤지암리636번지',
 '경기도 이천시 증일동',
 '경기도 광명시 소하동 380-1',
 '경기도 양주시 덕계동 707-1번지',
 '서울특별시 강동구 천호동 356',
 '경기도 평택시 합정동 835번지',


In [181]:
def get_lat_lon_kakao(address):
    kakao_api_url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {kakao_api_key}"}

    try:
        response = requests.get(kakao_api_url, headers=headers, params={"query": address})
        # 요청 실패한경우
        if response.status_code != 200:
            return None
        
        result = response.json()
        data = result['documents']
        has_data = len(data) != 0

        # 주소 데이터 없는경우
        if not has_data:
            print(f'{address} - 주소 데이터가 없습니다.')
            return None
        
        result = data[0]['address']

        region_depth_3_h = result['region_3depth_h_name'] # 동
        region_depth_3 = result['region_3depth_name'] # 동

        # 동 데이터 없는 경우
        if region_depth_3_h == '' and region_depth_3 == '':
            print(f'{address} - 동 데이터가 없습니다.')
            return None

        return result
    except Exception as e:
        print(f"Error fetching data for address {address}: {e}")
        return None

In [182]:
get_lat_lon_kakao('서울특별시 강동구')

서울특별시 강동구 - 동 데이터가 없습니다.


In [183]:
def process_address_data(address, address_by_apartname, verbose=False):
    """카카오맵 API를 사용해 위도, 경도 및 행정정보를 추출하는 함수"""
    
    try:
        # 1️⃣ 카카오맵 API 사용해서 주소 데이터 가져오기
        data = get_lat_lon_kakao(address)

        # 2️⃣ 공급위치로 주소 못 얻는 경우, 주택명 기반으로 재시도
        if not data:
            data = get_lat_lon_kakao(address_by_apartname)

        # 3️⃣ 그래도 없는 경우 → NaN 처리
        if not data:
            if verbose:
                print(f"[주소 변환 실패] '{address}', '{address_by_apartname}' → NaN 값 반환")
            return None, None, None, None, None, None, None, None  # 8개 반환 보장

        # 4️⃣ 정상적인 데이터 추출
        lat = data.get('y', None)
        lon = data.get('x', None)
        h_code = data.get('h_code', None)
        b_code = data.get('b_code', None)
        region_depth_1 = data.get('region_1depth_name', None)  # 도
        region_depth_2 = data.get('region_2depth_name', None)  # 시,구
        region_depth_3_h = data.get('region_3depth_h_name', None)  # 행정동
        region_depth_3 = data.get('region_3depth_name', None)  # 법정동

        # 5️⃣ 필요할 때만 로그 출력 (verbose=True 설정 시)
        if verbose:
            print(f"[주소 변환 성공] '{address}' → {lat}, {lon}, {h_code}, {b_code}, {region_depth_1}, {region_depth_2}, {region_depth_3_h}, {region_depth_3}")

        return lat, lon, h_code, b_code, region_depth_1, region_depth_2, region_depth_3_h, region_depth_3
    
    except Exception as e:
        print(f"[오류 발생] {e} (address='{address}', address_by_apartname='{address_by_apartname}')")
        return None, None, None, None, None, None, None, None  # 오류 발생 시도 안전하게 8개 반환


In [184]:
df[['위도', '경도', '행정동코드', '법정동코드', '시도', '시군구', '읍면동1', '읍면동2']] = df.apply(
    lambda x: process_address_data(x['정제된주소'], x['공급위치 by 주택명']), axis=1, result_type='expand'
)
df

인천광역시 미추홀구 숭의동 18번지 - 주소 데이터가 없습니다.
경기도 평택시 - 동 데이터가 없습니다.
인천광역시 미추홀구 학익동 321번지 - 주소 데이터가 없습니다.
인천광역시 서구 - 동 데이터가 없습니다.
경기도 안성시 공도읍 승두리 589-11번지 - 주소 데이터가 없습니다.
인천광역시 미추홀구 학익동 인천 용현·학익 - 주소 데이터가 없습니다.
인천검단시 서구 불로동 484-3번지 - 주소 데이터가 없습니다.
경기도 파주시 - 동 데이터가 없습니다.
경기 파주시 목동동 363 - 주소 데이터가 없습니다.
경기도 과천시 지식정보타운 - 주소 데이터가 없습니다.
경기도 군포시 - 동 데이터가 없습니다.
경기도 평택시 - 동 데이터가 없습니다.
인천광역시 서구 마전동 1158-12번지 - 주소 데이터가 없습니다.
인천 서구 마전동 1158-12 - 주소 데이터가 없습니다.
경기도 평택시 - 동 데이터가 없습니다.
경기도 평택시 현덕면 화양리 1로트 - 주소 데이터가 없습니다.
경기도 성남시 수정구 - 동 데이터가 없습니다.
nan - 주소 데이터가 없습니다.
경기도 평택시 - 동 데이터가 없습니다.
인천광역시 서구 - 동 데이터가 없습니다.
경기도 성남시 분당구 - 동 데이터가 없습니다.
경기 성남시 분당구 대장동 산 19-47 - 주소 데이터가 없습니다.
인천광역시 서구 - 동 데이터가 없습니다.
경기도 평택시 - 동 데이터가 없습니다.
경기도 안성시 - 동 데이터가 없습니다.
경기도 파주시 - 동 데이터가 없습니다.
인천광역시 미추홀구 숭의동 18번지 - 주소 데이터가 없습니다.
인천광역시 중구 영종하늘도시 - 주소 데이터가 없습니다.
경기도 파주시 - 동 데이터가 없습니다.
경기도 파주시 - 동 데이터가 없습니다.
경기도 양주시 - 동 데이터가 없습니다.
서울특별시 도봉구 도봉2동 95번지 - 주소 데이터가 없습니다.
서울 도봉구 도봉동 95 - 주소 데이터가 없습니다.
경기도 안양시 만안구 석수2동 348번지 - 주소 데이터가 없습니다.
경기도 

,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,...,공급위치 by 주택명,정제된주소,위도,경도,행정동코드,법정동코드,시도,시군구,읍면동1,읍면동2
0,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,경기 부천시 원미구 심곡동 465-11,경기도 부천시 원미구 신흥로 20,37.4853771294045,126.776922455931,4119252000,4119210200,경기,부천시 원미구,심곡2동,심곡동
1,2025000032,2025000032,성남판교 산운마을9단지 대방노블랜드 아파트 우선 분양전환 후 잔여세대(15세대),1,APT,3,국민,0,분양주택,410,...,NaN,경기도 성남시 분당구 산운로 126,37.39216864601,127.075563577376,4113568000,4113511500,경기,성남시 분당구,운중동,운중동
2,2025000020,2025000020,e편한세상 제물포역 파크메종(조합원 취소분),1,APT,1,민영,0,분양주택,400,...,인천 미추홀구 숭의동 18-1,인천광역시 미추홀구 숭의동 18번지,37.4610244665016,126.657793162727,2817753000,2817710100,인천,미추홀구,숭의4동,숭의동
3,2025000014,2025000014,평택청북 사랑으로 부영 2차 아파트 우선분양 전환 후 잔여세대,1,APT,1,민영,0,분양주택,410,...,경기 평택시 청북읍 옥길리 1104,경기도 평택시 청북읍 청북남로 277,37.0180254539746,126.91202301977,4122025900,4122025924,경기,평택시,청북읍,청북읍 옥길리
4,2025000013,2025000013,고양창릉지구 S-6블록 공공분양주택(본청약),1,APT,3,국민,0,분양주택,410,...,경기 고양시 덕양구 도내동 548,경기도 고양시 덕양구 원흥동,37.6475978173957,126.871008999634,,4128110400,경기,고양시 덕양구,,원흥동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,2020000303,2020000303,수원호매실14단지 B-3BL,1,APT,3,국민,1,분양전환 가능임대,410,...,경기 수원시 권선구 호매실동 1380,경기도 수원시 권선구 호매실로218번길 22,37.263205430795,126.958009297514,4111366400,4111313500,경기,수원시 권선구,호매실동,호매실동
849,2020000302,2020000302,수원호매실15단지 B-7BL,1,APT,3,국민,1,분양전환 가능임대,410,...,경기 수원시 권선구 호매실동 1388,경기도 수원시 권선구 호매실로165번길 70,37.2601436585013,126.952002712811,4111366400,4111313500,경기,수원시 권선구,호매실동,호매실동
850,2020000301,2020000301,수원호매실5단지 B-6BL,1,APT,3,국민,1,분양전환 가능임대,410,...,경기 수원시 권선구 금곡동 1081,경기도 수원시 권선구 금곡로 46,37.2714183047275,126.939250994367,4111366200,4111313400,경기,수원시 권선구,금곡동,금곡동
851,2020000304,2020000304,수원호매실13단지 A-4BL,1,APT,3,국민,1,분양전환 가능임대,410,...,경기 수원시 권선구 호매실동 1371,경기도 수원시 권선구 호매실로218번길 111,37.2655255050934,126.958210599618,4111366400,4111313500,경기,수원시 권선구,호매실동,호매실동


In [185]:
df[df['위도'].isna()]

,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,...,공급위치 by 주택명,정제된주소,위도,경도,행정동코드,법정동코드,시도,시군구,읍면동1,읍면동2
92,2024000278,2024000278,제일풍경채 운정,1,APT,1,민영,0,분양주택,410,...,경기 파주시 목동동 363,경기도 파주시,None,None,None,None,None,None,None,None
121,2024000106,2024000106,검단 스타힐스 가현숲,1,APT,1,민영,0,분양주택,400,...,인천 서구 마전동 1158-12,인천광역시 서구 마전동 1158-12번지,None,None,None,None,None,None,None,None
131,2024000154,2024000154,성남 복정1지구 B3블록 엘리프 남위례역 에듀포레(본청약),1,APT,1,민영,0,분양주택,410,...,NaN,경기도 성남시 수정구,None,None,None,None,None,None,None,None
153,2023000617,2023000617,판교TH212,1,APT,1,민영,0,분양주택,410,...,경기 성남시 분당구 대장동 산 19-47,경기도 성남시 분당구,None,None,None,None,None,None,None,None
196,2023000555,2023000555,도봉 금호어울림 리버파크,1,APT,1,민영,0,분양주택,100,...,서울 도봉구 도봉동 95,서울특별시 도봉구 도봉2동 95번지,None,None,None,None,None,None,None,None
229,2023000481,2023000481,오산세교2지구 A-13블록 호반써밋 본청약,1,APT,1,민영,0,분양주택,410,...,NaN,경기도 오산시,None,None,None,None,None,None,None,None
267,2023000252,2023000252,e편한세상 동탄 파크아너스 A56블럭(2회차),1,APT,1,민영,0,분양주택,410,...,NaN,경기도 화성시,None,None,None,None,None,None,None,None
301,2023000059,2023000059,e편한세상 동탄 파크아너스 A56블럭(1회차),1,APT,1,민영,0,분양주택,410,...,NaN,경기도 화성시,None,None,None,None,None,None,None,None
305,2023000040,2023000040,동인천역 파크푸르지오 10년 분양전환 공공임대주택,1,APT,3,국민,1,분양전환 가능임대,400,...,NaN,인천광역시 동구 송림동 185번지,None,None,None,None,None,None,None,None
331,2022000751,2022000751,파주 운정신도시 A2BL 호반써밋,1,APT,1,민영,0,분양주택,410,...,경기 파주시 당하동 426,경기도 파주시 당하동 파주운정,None,None,None,None,None,None,None,None


In [186]:
df.to_csv('청약 매물 주소변환.csv', index=False, encoding='cp949')

In [3]:
df_test = pd.read_csv('청약 매물 주소변환.csv', encoding='cp949')
df_test

,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,...,공급위치 by 주택명,정제된주소,위도,경도,행정동코드,법정동코드,시도,시군구,읍면동1,읍면동2
0,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,경기 부천시 원미구 심곡동 465-11,경기도 부천시 원미구 신흥로 20,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동
1,2025000032,2025000032,성남판교 산운마을9단지 대방노블랜드 아파트 우선 분양전환 후 잔여세대(15세대),1,APT,3,국민,0,분양주택,410,...,NaN,경기도 성남시 분당구 산운로 126,37.392169,127.075564,4.113568e+09,4.113512e+09,경기,성남시 분당구,운중동,운중동
2,2025000020,2025000020,e편한세상 제물포역 파크메종(조합원 취소분),1,APT,1,민영,0,분양주택,400,...,인천 미추홀구 숭의동 18-1,인천광역시 미추홀구 숭의동 18번지,37.461024,126.657793,2.817753e+09,2.817710e+09,인천,미추홀구,숭의4동,숭의동
3,2025000014,2025000014,평택청북 사랑으로 부영 2차 아파트 우선분양 전환 후 잔여세대,1,APT,1,민영,0,분양주택,410,...,경기 평택시 청북읍 옥길리 1104,경기도 평택시 청북읍 청북남로 277,37.018025,126.912023,4.122026e+09,4.122026e+09,경기,평택시,청북읍,청북읍 옥길리
4,2025000013,2025000013,고양창릉지구 S-6블록 공공분양주택(본청약),1,APT,3,국민,0,분양주택,410,...,경기 고양시 덕양구 도내동 548,경기도 고양시 덕양구 원흥동,37.647598,126.871009,NaN,4.128110e+09,경기,고양시 덕양구,NaN,원흥동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,2020000303,2020000303,수원호매실14단지 B-3BL,1,APT,3,국민,1,분양전환 가능임대,410,...,경기 수원시 권선구 호매실동 1380,경기도 수원시 권선구 호매실로218번길 22,37.263205,126.958009,4.111366e+09,4.111314e+09,경기,수원시 권선구,호매실동,호매실동
849,2020000302,2020000302,수원호매실15단지 B-7BL,1,APT,3,국민,1,분양전환 가능임대,410,...,경기 수원시 권선구 호매실동 1388,경기도 수원시 권선구 호매실로165번길 70,37.260144,126.952003,4.111366e+09,4.111314e+09,경기,수원시 권선구,호매실동,호매실동
850,2020000301,2020000301,수원호매실5단지 B-6BL,1,APT,3,국민,1,분양전환 가능임대,410,...,경기 수원시 권선구 금곡동 1081,경기도 수원시 권선구 금곡로 46,37.271418,126.939251,4.111366e+09,4.111313e+09,경기,수원시 권선구,금곡동,금곡동
851,2020000304,2020000304,수원호매실13단지 A-4BL,1,APT,3,국민,1,분양전환 가능임대,410,...,경기 수원시 권선구 호매실동 1371,경기도 수원시 권선구 호매실로218번길 111,37.265526,126.958211,4.111366e+09,4.111314e+09,경기,수원시 권선구,호매실동,호매실동


### 청약매물 목록 위도, 경도, 행정구역 컬럼 추가

In [ ]:
def lat_lon_pipeline(df):

    # 서울, 경기, 인천 지역만 필터링
    df = df[df['공급지역명'].isin(['서울', '경기', '인천'])]

    # 주택명 정제
    df_copy = preprocess_apartname(df)

    # Kakao API로 위도와 경도 가져오기
    df_copy = process_address_data(df_copy)
    
    # 주소 분리 및 업데이트 
    df_copy = process_address(df_copy)

    # 위도와 경도가 None인 행만 선택하여 다시 처리
    df_none = df_copy[(df_copy['위도'].isnull()) & (df_copy['경도'].isnull())].copy()

    len(df_none)

    if not df_none.empty:
        df_none_processed = process_address_data(df_none)
        df_copy.loc[df_none.index, ['위도', '경도']] = df_none_processed[['위도', '경도']]
    
    df_copy.drop("정제_공급위치", axis=1, inplace=True)
    
    return df, df_copy


In [126]:
df = pd.read_csv("병합_청약매물_목록_정보.csv", encoding='cp949')
df, df_copy = lat_lon_pipeline(df)
df_merged = pd.merge(df, df_copy, on='공고번호', how='left')

/var/folders/pl/c9208lms3f937xrsflvwrm640000gn/T/ipykernel_67897/785735821.py:1: DtypeWarning: Columns (36,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("병합_청약매물_목록_정보.csv", encoding='cp949')


Error fetching data for address 인천광역시 미추홀구 숭의동 18번지: list index out of range
Error fetching data for address 인천광역시 미추홀구 숭의동 18번지: list index out of range
Error fetching data for address 인천광역시 미추홀구 숭의동 18번지: list index out of range
Error fetching data for address 인천광역시 미추홀구 숭의동 18번지: list index out of range
Error fetching data for address 인천광역시 미추홀구 숭의동 18번지: list index out of range
Error fetching data for address 인천광역시 미추홀구 숭의동 18번지: list index out of range
Error fetching data for address 인천광역시 미추홀구 숭의동 18번지: list index out of range
Error fetching data for address 인천광역시 미추홀구 숭의동 18번지: list index out of range
Error fetching data for address 인천광역시 미추홀구 숭의동 18번지: list index out of range
Error fetching data for address 인천광역시 미추홀구 숭의동 18번지: list index out of range
Error fetching data for address 인천광역시 미추홀구 숭의동 18번지: list index out of range
Error fetching data for address 인천광역시 미추홀구 숭의동 18번지: list index out of range
Error fetching data for address 인천광역시 미추홀구 숭의동 18번지: list index out of range

ValueError: not enough values to unpack (expected 8, got 2)

In [5]:
df_merged.to_csv('병합_청약매물_목록_정보_위도경도추가.csv', index=False, encoding='cp949')

In [3]:
df_merged = pd.read_csv("병합_청약매물_목록_정보_위도경도추가.csv", encoding='cp949')

/var/folders/pl/c9208lms3f937xrsflvwrm640000gn/T/ipykernel_91358/1948573066.py:1: DtypeWarning: Columns (36,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_merged = pd.read_csv("병합_청약매물_목록_정보_위도경도추가.csv", encoding='cp949')


In [5]:
df_merged

,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,...,접수건수,경쟁률,최저당첨가점,최고당첨가점,평균당첨가점,위도,경도,시도,시군구,읍면동
0,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,0,(△5),NaN,NaN,NaN,37.485377,126.776922,경기도,부천시,원미구
1,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,15,3.00,NaN,NaN,NaN,37.485377,126.776922,경기도,부천시,원미구
2,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,3,-,NaN,NaN,NaN,37.485377,126.776922,경기도,부천시,원미구
3,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,10,-,NaN,NaN,NaN,37.485377,126.776922,경기도,부천시,원미구
4,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,1,(△4),NaN,NaN,NaN,37.485377,126.776922,경기도,부천시,원미구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21226,2020000305,2020000305,수원세류2단지 2BL,1,APT,3,국민,1,분양전환 가능임대,410,...,0,NaN,NaN,NaN,NaN,37.263418,127.006154,경기도,수원시,권선구
21227,2020000305,2020000305,수원세류2단지 2BL,1,APT,3,국민,1,분양전환 가능임대,410,...,0,NaN,NaN,NaN,NaN,37.263418,127.006154,경기도,수원시,권선구
21228,2020000305,2020000305,수원세류2단지 2BL,1,APT,3,국민,1,분양전환 가능임대,410,...,0,NaN,NaN,NaN,NaN,37.263418,127.006154,경기도,수원시,권선구
21229,2020000305,2020000305,수원세류2단지 2BL,1,APT,3,국민,1,분양전환 가능임대,410,...,0,NaN,NaN,NaN,NaN,37.263418,127.006154,경기도,수원시,권선구
